## CGM Coordinates

Convert station coordinates from latitude and longitude to altitude adjusted corrected geomagnetic coordinates.

This is written as a notebook instead of apart of utils.py as it requires IGRF12 and AACGMv2 which can be tricky to install (at least in windows).

- cgm lat
- cmg lon
- mlt at 0 UT
- declination
- L-shell dipole
- L-shell Geopack IGRF

In [16]:
import igrf12
import aacgmv2
import pandas as pd
import numpy as np
import datetime as datetime

In [17]:
year   = np.arange(1990,2000,1)
alt_km = 100.
stn_f = "..\\gmag\\Stations\\station_list.csv"
year

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999])

In [18]:
# loop through the years and calculate geomagnetic coordinates for all stations
for yr in year:
    dt = pd.to_datetime(str(yr))
    
    df = pd.read_csv(stn_f, header=None, skiprows=1, 
                     names = ['array','code','name','latitude','longitude'])

    decl    = np.zeros(df.shape[0])
    cgm_lat = np.zeros(df.shape[0])
    cgm_lon = np.zeros(df.shape[0])
    l_dip   = np.zeros(df.shape[0])
    mlt     = np.zeros(df.shape[0])
    mlt_ut  = np.zeros(df.shape[0])


    # get declination, cgm coords, l-shell
    for index, row in df.iterrows():

        mag = igrf12.igrf(dt,glat=row['latitude'],glon=row['longitude'],alt_km=alt_km)

        cgm_lat[index], cgm_lon[index], cgm_r = aacgmv2.convert_latlon(row['latitude'],row['longitude'], alt_km,dt,code='G2A')

        if cgm_lon[index] <0:
            cgm_lon[index] = 360+cgm_lon[index]

        mlt_ut[index] = aacgmv2.convert_mlt(cgm_lon[index],datetime.datetime(int(yr),1,1,0,0,0),m2a=False)
        mlt[index] = 24- mlt_ut[index]

        l_dip[index] = 1./(np.cos(np.deg2rad(cgm_lat[index]))**2.)

        decl[index] = mag['decl'].values[0]
        if row['code'] == 'GILL':
            print(yr,index,decl[index],cgm_lat[index],cgm_lon[index],l_dip[index])

    df['cgm_latitude']  = cgm_lat
    df['cgm_longitude'] = cgm_lon
    df['declination']   = decl
    df['lshell']        = l_dip
    df['mlt_midnight']  = mlt
    df['mlt_ut']        = mlt_ut

    fn = '..\\gmag\\Stations\\{0:04d}_station_cgm.txt'.format(int(yr))
    df.to_csv(fn,index=False,float_format="%E", na_rep='NaN')

1990 9 2.337929259736139 67.03473883891932 330.43947053941963 6.568794717149296
1991 9 2.218997933272899 66.99625690255634 330.588037792463 6.548024677096112
1992 9 2.1009167972184803 66.95760090942568 330.73613466558385 6.527265633835043
1993 9 1.983677759941038 66.9187714672727 330.88376141965773 6.506518623768709
1994 9 1.8672728166005346 66.87976918172059 331.03091833887464 6.485784668884457
1995 9 1.7516940020425356 66.84059465623507 331.17760573041267 6.465064776796083
1996 9 1.6046430355238082 66.78578542694908 331.3052184351593 6.436251805617153
1997 9 1.4591762005958187 66.73083229104805 331.4322624113592 6.407567874950899
1998 9 1.3152698817844586 66.67573583106059 331.5587402115949 6.3790129603923695
1999 9 1.1729008888165 66.62049662379138 331.68465438781976 6.350587022805419


## Examples

Simple examples exploring the DataFrame containing station geomagnetic coordinates. 

In [9]:
# print the station DataFrame
df.head()

,array,code,name,latitude,longitude,cgm_latitude,cgm_longitude,declination,lshell,mlt_midnight,mlt_ut
9,CARISMA,GILL,Gillam,56.376,265.36,67.623292,327.5121,5.444329,6.899964,7.324378,16.675622


In [5]:
# Get only the Gill entry
df[df['code'] == 'GILL']

,array,code,name,latitude,longitude,cgm_latitude,cgm_longitude,declination,lshell,mlt_midnight,mlt_ut
9,CARISMA,GILL,Gillam,56.376,265.36,66.338259,332.397816,0.483748,6.20845,7.007026,16.992974


In [6]:
# Get all entryies between L of 3 and 7
# list the array, code, cgm longitude and l-shell
# allow pandas to print all output
pd.set_option('display.max_rows', None) 
df[(df['lshell']>3) & (df['lshell']<7)][['array','code','cgm_longitude','lshell']]

,array,code,cgm_longitude,lshell
1,CARISMA,BACK,332.853473,6.905567
3,CARISMA,DAWS,273.158896,6.024881
5,CARISMA,FCHP,307.802158,6.251058
7,CARISMA,FSIM,293.502147,6.754608
8,CARISMA,FSMI,306.119254,6.813183
9,CARISMA,GILL,332.397816,6.208450
10,CARISMA,GULL,314.323281,3.649343
11,CARISMA,ISLL,332.729247,5.179275
12,CARISMA,LGRR,331.761063,4.572246
13,CARISMA,MCMU,308.497652,5.341763


In [7]:
# Get all stations with L greater than 7
df[(df['lshell']>7)][['array','code','cgm_longitude','lshell']]

,array,code,cgm_longitude,lshell
2,CARISMA,CONT,303.657802,11.716480
4,CARISMA,ESKI,332.426661,9.281806
6,CARISMA,FCHU,332.864121,7.537080
15,CARISMA,NORM,284.838648,8.263808
21,CARISMA,RANK,335.250151,11.128364
22,CARISMA,SACH,278.934676,17.576047
23,CARISMA,TALO,330.105099,25.607975
28,CANMOS,ALE,91.838269,383.914603
29,CANMOS,BLC,328.217614,12.648211
31,CANMOS,CBB,310.079354,19.879246
